# Answering Business Questions Using SQL
The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|      Artist Name     	|  Genre  	|
|:--------------------:	|:-------:	|
| Regal                	| Hip-Hop 	|
| Red Tone             	| Punk    	|
| Meteor and the Girls 	| Pop     	|
| Slim Jim Bites       	| Blues   	|

### Goal
The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


![schema](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg  "Schema diagram")

Above is the schema diagram of the database we are working with.

In [3]:
%%sql
SELECT *
FROM invoice_line 
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [4]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
    
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id

GROUP BY 1
ORDER BY 2 DESC

LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


`Rock` is the the most popularly music genre in the US. But we have to check among the genres of the selected artists. `alternatives and punk` is the most popular among the genres of the artist.

## Best Performing Sales Support Agent
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [5]:
%%sql

WITH employee_customer AS
   (
    SELECT e.*, i.* FROM employee e
    INNER JOIN customer c on c.support_rep_id = e.employee_id
    INNER JOIN invoice i on i.customer_id = c.customer_id
   )
    
SELECT
    employee_id,
    ec.first_name || " " || ec.last_name employee,
    title,
    ROUND(TOTAL(total), 2) total_amount_sales_dollars, 
    hire_date
    FROM employee_customer ec
GROUP BY 1
ORDER BY 4 DESC;

 * sqlite:///chinook.db
Done.


employee_id,employee,title,total_amount_sales_dollars,hire_date
3,Jane Peacock,Sales Support Agent,1731.51,2017-04-01 00:00:00
4,Margaret Park,Sales Support Agent,1584.0,2017-05-03 00:00:00
5,Steve Johnson,Sales Support Agent,1393.92,2017-10-17 00:00:00


`Jane Peacock` seems to be the best performing sale's support agent based on total sales amount made. However, The other two sales support agent were hired at a later date than Jane.

## Analyzing Sales by Country
In particular, chinook wants to take a look at the data for each country, on the following:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [35]:
%%sql
WITH customer_by_region AS
   (
    SELECT c.*, i.total, il.quantity  FROM customer c
    INNER JOIN invoice i on i.customer_id = c.customer_id
    INNER JOIN invoice_line il on i.invoice_id = il.invoice_id
    
   )
    
    
SELECT 
country1 country, 
no_of_orders,
total_customers no_of_customers,
total_sales_amount total_sales,
ave_orders,
ROUND(ave_sales, 2)
FROM
    (
    SELECT
        cr.*,
        CASE
            WHEN cr.no_of_cust = 1 THEN 'others'
            ELSE cr.country
    
        END AS country1, 
        COUNT(cr.no_of_cust) AS total_customers
    FROM customer_region cr
    GROUP BY country1
        ORDER BY total_sales_amount DESC,
        CASE WHEN country = 'other' THEN 1 ELSE 0 END DESC
   )



 * sqlite:///chinook.db
Done.


country,no_of_orders,no_of_customers,total_sales,ave_orders,"ROUND(ave_sales, 2)"
USA,1051,1,10405.89,80,800.45
Canada,541,1,5489.55,67,686.19
Brazil,432,1,4059.0,86,811.8
France,393,1,3972.87,78,794.57
Germany,338,1,3441.24,84,860.31
Czech Republic,276,1,3183.84,138,1591.92
United Kingdom,248,1,2498.76,82,832.92
India,185,1,1887.93,92,943.97
Portugal,187,1,1755.27,93,877.64
others,116,15,1433.52,116,1433.52


USA is the country with the most sales and customers. There are a few countries with only one customer. We have grouped these countries together.

## Track or Album: Determining Purchase Type
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [12]:
     
     %%sql

WITH invoice_albums AS
    (
        SELECT
        i.invoice_id,
        COUNT(t.track_id) purchased_track_count,
        COUNT(DISTINCT t.album_id) album_count,
        t.album_id album_id
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        INNER JOIN track t ON t.track_id = il.track_id
        GROUP BY 1
    ),
    album_track_count AS
    (
        SELECT
        t.album_id album_id,
        COUNT(t.track_id) album_track_count
        FROM track t
        GROUP BY 1
    )
    

SELECT  
    CASE
        WHEN ia.album_count = 1
        AND ia.purchased_track_count = atc.album_track_count
        THEN "Albums"
        ELSE "Singles"
        END AS purchase_type,
    COUNT(ia.invoice_id) number_of_invoices,
    ROUND(CAST(COUNT(ia.invoice_id) AS FLOAT) / 
          (
              SELECT COUNT(*)
              FROM invoice), 2) percentage
FROM invoice_albums ia
INNER JOIN album_track_count atc ON atc.album_id = ia.album_id
GROUP BY purchase_type

LIMIT 10;

 * sqlite:///chinook.db
Done.


purchase_type,number_of_invoices,percentage
Albums,114,0.19
Singles,500,0.81


Album purchases account for 19% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.

# Protected Media Types VS Non-protected Media Types
We also want to look into the protection status of the tracks and how it affects popularity and sales.

In [77]:
%%sql
select 
    protection_status,
    COUNT(quantity) quantity,
    ROUND(TOTAL(total)) total

FROM
(SELECT *,
 CASE
 WHEN m.name LIKE "%PROTECTED%" THEN "PROTECTED"
    ELSE "NOT PROTECTED"
    END AS protection_status
  FROM
    media_type m 
INNER JOIN track t on t.media_type_id = m.media_type_id
INNER JOIN invoice_line il on t.track_id = il.track_id
INNER JOIN invoice i on il.invoice_id = i.invoice_id)
GROUP by 1;

 * sqlite:///chinook.db
Done.


protection_status,quantity,total
NOT PROTECTED,4315,43362.0
PROTECTED,442,4141.0


We see that non-protected media types are by far more popular and make more sales that protected media types.